In [ ]:
!pip install pandas matplotlib seaborn scikit-learn numpy

In [ ]:
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.linear_model import LinearRegression
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.ensemble import GradientBoostingRegressor
import numpy as np
from scipy import stats
from scipy.stats import norm, skew

In [ ]:
df = pd.read_csv("data/lego_sets.csv")

In [ ]:
df

In [ ]:

# Remove duplicate rows
df.drop_duplicates(inplace=True)

# Handle missing values in other columns
df['Pricing'] = df['Pricing'].fillna(0)

# Convert "Details" column to string data type
df['Details'] = df['Details'].astype(str)

# Standardize currency values to USD
# Ensure the column containing currency values exists and has the correct name
if 'Pricing' in df.columns:
    # Remove currency symbols and any other non-numeric characters
    df['Pricing'] = df['Pricing'].str.replace('[^\d.]', '', regex=True)
    # Convert to float
    df['Pricing'] = pd.to_numeric(df['Pricing'], errors='coerce')


if 'Set Predictions' in df.columns:
    df['Set Predictions'] = df['Set Predictions'].astype(str)
    df['Set Predictions'].fillna('', inplace=True)
    df['Set Predictions'] = df['Set Predictions'].str.replace('[^\d.]', '', regex=True)
    df['Set Predictions'] = pd.to_numeric(df['Set Predictions'], errors='coerce')

# Drop irrelevant columns
df.drop(['Subtheme Analysis'], axis=1, inplace=True)

# Rename columns for clarity
df.rename(columns={'Details': 'Set Number', 'Set Facts': 'Facts'}, inplace=True)

# Remove leading and trailing whitespace from the "Facts" column
df['Facts'] = df['Facts'].str.strip().replace('\n', '', regex=True)

# Replace non-numeric characters in "Quick Buy" with empty strings
if 'Quick Buy' in df.columns and df['Quick Buy'].dtype == 'object':
    df['Quick Buy'] = df['Quick Buy'].str.replace('[^\d.]', '', regex=True)

# Convert "Quick Buy" column to numeric, coercing errors to NaN
if 'Quick Buy' in df.columns:
    df['Quick Buy'] = pd.to_numeric(df['Quick Buy'], errors='coerce')

# Drop rows with NaN values in the "Quick Buy" column
df.dropna(subset=['Quick Buy'], inplace=True)

# Find the Lego set with the lowest price in the "Quick Buy" section
if 'Quick Buy' in df.columns:
    lowest_price_set = df.loc[df['Quick Buy'].idxmin()]

# Display the DataFrame
df

In [ ]:
# Display the Lego set with the lowest price
print("Lego set with the lowest price in the 'Quick Buy' section:")
lowest_price_set

In [ ]:
highest_price_set = df.loc[df['Quick Buy'].idxmax()]
print("Lego set with the highest price in the 'Quick Buy' section:")
highest_price_set

In [ ]:
# Find the average price of all Lego sets in the "Pricing" section
average_price = df['Pricing'].mean()
print(f"The average price of all Lego sets in the 'Pricing' section is ${average_price:.2f}.")
average_price

In [ ]:
df_0011 = pd.read_csv("data/001-1_history.csv")

In [ ]:
df_0011

In [ ]:
# Step 1: Remove duplicate rows
df_0011.drop_duplicates(inplace=True)

# Step 2: Handle missing values in the 'Currency Value' column
df_0011['Currency Value'] = df_0011['Currency Value'].fillna(0)

# Step 3: Standardize currency values to USD
# Remove currency symbols and any other non-numeric characters
df_0011['Currency Value'] = df_0011['Currency Value'].str.replace('[^\d.]', '', regex=True)
# Convert to float
df_0011['Currency Value'] = pd.to_numeric(df_0011['Currency Value'], errors='coerce')

# Step 4: Drop unnecessary columns
df_0011.drop(['Value', 'Status'], axis=1, inplace=True)

df_0011['Date'] = pd.to_datetime(df_0011['Date'], errors='coerce')

# Step 5: Rename columns for clarity
df_0011.rename(columns={'Currency Value': 'USD Value', 'Description': 'Set Description'}, inplace=True)

# remove NAN values
df_0011['USD Value'] = df_0011['USD Value'].fillna(0)

df_0011 = df_0011[df_0011['USD Value'] != 0]

# remove nan values description remplace by ''
df_0011['Set Description'] = df_0011['Set Description'].fillna('')

df_0011

In [ ]:
X = df_0011['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
y = df_0011['USD Value'].values

# Fit the model
model = LinearRegression().fit(X, y)

# Prepare future dates and predict
future_dates = pd.date_range('2008-12-31', '2030-12-31', freq='Y')
future_dates_ordinal = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
future_predictions = model.predict(future_dates_ordinal)

# Plotting, making sure to convert ordinal dates back to datetime for readability
plt.figure(figsize=(10, 6))
plt.scatter(df_0011['Date'], df_0011['USD Value'], color='blue', label='Data points')
plt.plot(df_0011['Date'], model.predict(X), color='red', label='Linear regression')
plt.plot(future_dates, future_predictions, color='green', label='Future predictions', linestyle='--')

plt.title('Linear Regression of Lego Set 001-1')
plt.xlabel('Date')
plt.ylabel('USD Value')
plt.xticks(rotation=45)
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
def monte_carlo_simulation(data, days, iterations):
    # Convert to numpy array if data is a pandas Series
    if isinstance(data, pd.Series):
        data = data.to_numpy()

    # Ensure data is a numpy array
    if not isinstance(data, np.ndarray):
        raise TypeError("Data must be a numpy array or pandas Series")

    # Calculate log returns
    log_returns = np.log(data[1:] / data[:-1])
    mean = np.mean(log_returns)
    variance = np.var(log_returns)
    
    # Adjust the drift to use a smaller portion of the variance
    drift = mean - (0.05 * variance)
    daily_volatility = np.std(log_returns)

    future_prices = np.zeros((days, iterations))
    current_price = data[-1]
    for t in range(days):
        shocks = drift + daily_volatility * norm.ppf(np.random.rand(iterations))
        future_prices[t] = current_price * np.exp(shocks)
        current_price = future_prices[t]
    return future_prices

In [ ]:
simulation_days = 3
mc_iterations = 5

mc_prices = monte_carlo_simulation(df_0011['USD Value'], simulation_days, mc_iterations)

last_close_price = df_0011['USD Value'].iloc[-1]
mc_prices = np.vstack([np.full(mc_iterations, last_close_price), mc_prices])

last_date = df_0011['Date'].iloc[-1]
simulation_dates = pd.date_range(start=last_date, periods=simulation_days + 1, freq='D')

plt.figure(figsize=(14, 7))
plt.plot(df_0011['Date'], df_0011['USD Value'], color='blue', label='Historical prices')
for i in range(mc_iterations):
    plt.plot(simulation_dates, mc_prices[:, i], color='grey', alpha=0.1)
mean_mc_path = np.mean(mc_prices, axis=1)
plt.plot(simulation_dates, mean_mc_path, color='red', linewidth=2, label='Mean Future Path')

plt.title('Monte Carlo Simulation of Lego Set 001-1 Prices')
plt.xlabel('Date')
plt.ylabel('USD Value')

In [ ]:
X = df_0011['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
y = df_0011['USD Value'].values

future_dates = pd.date_range('2008-12-31', '2030-12-31', freq='Y')
future_dates_ordinal = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
future_predictions = model.predict(future_dates_ordinal)

# Gradient Boosting Regression
model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=2, random_state=0, loss='squared_error').fit(X, y)
print(model.score(X, y))
plt.figure(figsize=(10, 6))
plt.scatter(df_0011['Date'], df_0011['USD Value'], color='blue', label='Data points')
plt.plot(df_0011['Date'], model.predict(X), color='red', label='Gradient Boosting Regression')
plt.plot(future_dates, model.predict(future_dates_ordinal), color='green', label='Future predictions', linestyle='--')

plt.title('Gradient Boosting Regression')
plt.xlabel('Index')
plt.ylabel('USD Value')
plt.legend()
plt.grid(True)
plt.tight_layout()
plt.show()

In [ ]:
# sort by date
df_0011.sort_values(by=['Date'], inplace=True)

# calculate gain per year
df_0011['Gain'] = df_0011.groupby(df_0011['Date'].dt.year)['USD Value'].pct_change()

df_0011.dropna(subset=['Gain'], inplace=True)

df_0011

In [ ]:
# Plot the gain over time
plt.figure(figsize=(10, 6))
plt.plot(df_0011['Date'], df_0011['Gain'], marker='o', linestyle='-', color='b')
plt.title('Gain Over Time')
plt.xlabel('Date')
plt.ylabel('Gain')
plt.grid(True)
plt.xticks(rotation=45)
plt.tight_layout()
plt.show()


In [ ]:
# Create subplots
fig, axes = plt.subplots(nrows=2, ncols=1, figsize=(12, 12))

# Plot 1: Gain Over Time
axes[0].plot(df_0011['Date'], df_0011['Gain'], marker='o', linestyle='-', color='b', label='Gain')
axes[0].set_title('Gain Over Time')
axes[0].set_xlabel('Date')
axes[0].set_ylabel('Gain')
axes[0].grid(True)
axes[0].legend()

# Add trend line to the first plot
slope, intercept, r_value, p_value, std_err = stats.linregress(df_0011.index, df_0011['Gain'])
axes[0].plot(df_0011['Date'], intercept + slope * df_0011.index, color='red', label='Trend line')

# Annotate the trend line equation
axes[0].text(df_0011['Date'].iloc[5], df_0011['Gain'].iloc[5], f'y = {slope:.2f}x + {intercept:.2f}', fontsize=10, color='black')

# Rotate x-axis labels for better readability
axes[0].tick_params(axis='x', rotation=45)

# Annotate the maximum gain if df_0011 is not empty
if not df_0011.empty:
    max_gain_index = df_0011['Gain'].idxmax()
    if not pd.isna(max_gain_index):
        max_gain_date = df_0011.loc[max_gain_index, 'Date']
        max_gain_value = df_0011.loc[max_gain_index, 'Gain']
        axes[0].annotate('Max Gain', xy=(max_gain_date, max_gain_value),
                         xytext=(max_gain_date - pd.Timedelta(days=100), max_gain_value + 0.5),
                         arrowprops=dict(facecolor='black', arrowstyle='->'))

# Highlight points of interest if df_0011 is not empty
if not df_0011.empty:
    points_of_interest_index = [5, 11, 13, 23]
    points_of_interest = df_0011.iloc[points_of_interest_index]
    axes[0].scatter(points_of_interest['Date'], points_of_interest['Gain'], color='red', label='Points of Interest')

# Plot 2: USD Value Over Time
axes[1].scatter(df_0011['Date'], df_0011['USD Value'], color='blue', label='Data points')
axes[1].set_title('USD Value Over Time')
axes[1].set_xlabel('Date')
axes[1].set_ylabel('USD Value')
axes[1].legend()
axes[1].grid(True)

# Linear Regression for USD Value over Time
X = df_0011['Date'].astype(int).values.reshape(-1, 1)  # Convert datetime to numerical representation
y = df_0011['USD Value'].values
model = LinearRegression().fit(X, y)
axes[1].plot(df_0011['Date'], model.predict(X), color='red', label='Linear regression')

# Rotate x-axis labels for better readability
for ax in axes:
    ax.tick_params(axis='x', rotation=45)

plt.tight_layout()
plt.show()

In [ ]:
df_0011_history = pd.read_csv("data/001-1_new.csv")

In [ ]:
df_0011_history

In [ ]:
# Plot the model
plt.figure(figsize=(10, 6))
plt.scatter(df_0011_history.Date, df_0011_history['Value 1'], color='red', label='Lowest Price')
plt.scatter(df_0011_history.Date, df_0011_history['Value 2'], color='yellow', label='Medium Price')
plt.scatter(df_0011_history.Date, df_0011_history['Value 3'], color='blue', label='Above Average Price')
plt.scatter(df_0011_history.Date, df_0011_history['Value 4'], color='green', label='Highest Price')


plt.title('Value Over Time')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.model_selection import train_test_split
import numpy as np
import pandas as pd

X = df_0011_history['Date'].map(pd.Timestamp.toordinal).values.reshape(-1, 1)
y = df_0011_history['Value 1'].values

future_dates = pd.date_range('2010-12-31', '2030-12-31', freq='Y')
future_dates_ordinal = future_dates.map(pd.Timestamp.toordinal).values.reshape(-1, 1)
future_predictions = model.predict(future_dates_ordinal)

# Gradient Boosting Regression
model = GradientBoostingRegressor(n_estimators=1000, learning_rate=0.1, max_depth=2, random_state=0, loss='squared_error').fit(X, y)
print(model.score(X, y))
plt.figure(figsize=(10, 6))
plt.scatter(df_0011_history.Date, df_0011_history['Value 1'], color='blue', label='Data points')
plt.plot(df_0011_history.Date, model.predict(X), color='red', label='Gradient Boosting Regression')
plt.plot(future_dates, model.predict(future_dates_ordinal), color='green', label='Future predictions', linestyle='--')

plt.title('Value Over Time with Future Predictions')
plt.xlabel('Date')
plt.ylabel('Value')
plt.legend()
plt.grid(True)
plt.show()

In [ ]:
df_candle = df_0011_history[['Date', 'Value 1', 'Value 2', 'Value 3', 'Value 4']].copy()
df_candle.columns = ['Date', 'Lowest Price', 'Medium Price', 'Above Average Price', 'Highest Price']


df_candle['Date'] = pd.to_datetime(df_candle['Date'], errors='coerce')
df_candle.sort_values('Date', inplace=True)

# Plotting each 'candle'

plt.figure(figsize=(15, 8))

# Generating date ticks for cleaner x-axis
date_ticks = pd.date_range(start=df_candle['Date'].min(), end=df_candle['Date'].max(), periods=10)
plt.xticks(date_ticks, [date.strftime('%Y-%m-%d') for date in date_ticks], rotation=45)

# Define colors for different parts of the candle
color_high = 'green'
color_above_average = 'blue'
color_medium = 'yellow'
color_low = 'red'



# Plotting lines and markers for each 'candle'
for i in range(len(df_candle)):
    # Plot the high and low values
    plt.plot([i, i], [df_candle['Lowest Price'].iloc[i], df_candle['Highest Price'].iloc[i]], color='black')
    plt.plot([i, i], [df_candle['Above Average Price'].iloc[i], df_candle['Medium Price'].iloc[i]], color='black')
    plt.scatter(i, df_candle['Lowest Price'].iloc[i], color=color_low, marker='o')
    plt.scatter(i, df_candle['Highest Price'].iloc[i], color=color_high, marker='o')

plt.title('Simplified Candlestick Chart')
plt.xlabel('Date')
plt.ylabel('Price')
plt.grid(True)
plt.tight_layout()
plt.show()